In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=MNIST/seed=1_epoch=1_lr=0.001_alpha=0.0_tau=0.0


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]


In [3]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
import copy
mem = copy.deepcopy(benchmark.trains)
mem[1].dataset = copy.deepcopy(mem[1].dataset)
original_shape = mem[1].dataset.data.shape
mem[1].dataset.data = torch.zeros([0, *original_shape[1:]], dtype=torch.uint8)
mem[1].dataset.targets = torch.zeros([0], dtype=int)
mem[1].selected_indices = np.zeros([0], dtype=int)
mem[1].targets = np.zeros([0], dtype=int)
mem[1].sample_weight = torch.ones([0])
    

def add_data(self, X, y, weight=None):
    if not X.shape[0] == y.shape[0]:
        raise ValueError(f"Wrong size: {X.shape=}, {y.shape=}")
    if weight is None:
        weight = torch.ones_like(y)
    else:
        if not weight.shape == y.shape:
            ValueError(f"Wrong size: {X.shape=}, {y.shape=}, {weight.shape=}") 
    original_dataset_len = len(self.dataset)
    self.dataset.data = torch.cat([self.dataset.data, X], dim=0)
    self.dataset.targets = torch.cat([self.dataset.targets, y], dim=0)
    self.targets = np.concatenate([self.targets, y], axis=0)
    append_idx = np.arange(original_dataset_len, original_dataset_len+len(y), dtype=int)
    self.selected_indices = np.concatenate([self.selected_indices, append_idx], axis=0)
    self.sample_weight = torch.cat([self.sample_weight, weight], dim=0)

def replace_data(self, X, y, idx, weight=None):
    if not X.shape[0] == y.shape[0]:
        raise ValueError(f"Wrong size: {X.shape=}, {y.shape=}")
    if weight is None:
        weight = torch.ones_like(y, dtype=torch.float32)
    else:
        if not weight.shape == y.shape:
            ValueError(f"Wrong size: {X.shape=}, {y.shape=}, {weight.shape=}") 
    self.dataset.data[idx] = X.clone()
    self.dataset.targets[idx] = y.clone()
    self.targets[idx] = y.clone().cpu().numpy()
    self.sample_weight[idx] = weight.clone()

In [5]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
solver=<function LS_solver at 0x7f6040385160>
[1] Eval metrics for task 1 >> {'accuracy': 99.71478018520183, 'loss': 5.772249579394399e-05, 'std': 0.020902634181430013}
training_task_end
---------------------------- Task 2 -----------------------
solver=<function LS_solver at 0x7f6040385160>
[2] Eval metrics for task 1 >> {'accuracy': 0.0, 'loss': 0.02555385977382074, 'std': 0.0}
[2] Eval metrics for task 2 >> {'accuracy': 96.04392125259038, 'loss': 0.0004548145408845205, 'std': 0.9857817177066519}
training_task_end
---------------------------- Task 3 -----------------------
solver=<function LS_solver at 0x7f6040385160>
[3] Eval metrics for task 1 >> {'accuracy': 0.0, 'loss': 0.03241517470519875, 'std': 0.0}
[3] Eval metrics for task 2 >> {'accuracy': 0.0, 'loss': 0.031084835470948702, 'std': 0.0}
[3] Eval metrics for task 3 >> {'accuracy': 97.41170668444559, 'loss': 0.00038423587189412805, 'std': 1.6717963705442374}
training_

In [6]:
metric_manager_callback.meters['std'].get_std()

[0.020902634181430013,
 48.027019324448275,
 45.93046182082289,
 42.888841573207785,
 38.48515298280048]

In [7]:
metric_manager_callback.meters['std'].get_data()

array([[0.021, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.986, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 1.672, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.117, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.319]])

In [8]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.715,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , 96.044,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   , 97.412,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , 99.047,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , 96.212]])

In [9]:
metric_manager_callback.meters['accuracy'].compute_overall()

44.84232299873832

In [10]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"fairness:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:44.84232299873832
fairness:35.070475667092175


In [11]:
metric_manager_callback.meters['std'].compute_overall()

35.070475667092175